### Import libraries

Import libraries, set parameters, and import classes

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
np.random.seed(1)
tf.random.set_seed(1)
tf.keras.backend.set_floatx('float32')

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3,3)
plt.rcParams.update({'font.size': 13})

In [ ]:
from __lib__.TANN_class import TANN
from __lib__.DataProcessing import PreProcessing

### TANN

Import trained model by specifying the file (filename) of the saved weights and the architecture.
The function import_compile(filename,input_shape) performs a check of the defined parameters wrt the saved model

In [ ]:
# file where the model was saved
filename = './results/TANN_Elasticity'


# build network, import weights, and compile to make predictions
umat_dim = 4 # dimensional space
hidden_NN_Energy = 4 # number hidden layers in NN Energy
material = 'elastic'

TANNnet = TANN(umat_dim,
               material,
               hidden_NN_Energy,
               activation_NN_Energy='fast_gelu') # Initialize TANN class

# build TF model class
input_shape = [None,umat_dim]

TANNnet.import_compile(filename,input_shape)


#### Inference: loading path

Make predictions of the constitutive response under a simple shear loading path using the trained model. The loading path is predefined (see below), but you have to implement a function that call the trained model and make predictions.

Note. All quantities should be physical (not normalized)

In [ ]:
increments = 30
sub_increments = increments//2
de = 1e-3
umat_dir = -1

dstrain = np.zeros((increments,umat_dim))
dstrain[:sub_increments,umat_dir] = de*np.ones((sub_increments))
dstrain[sub_increments:2*sub_increments,umat_dir] = -de*np.ones((sub_increments))

strain = np.zeros((1,umat_dim))

#### Make predictions

Here goes your function to predict the stress and internal variables deriving from the deformation path specified by dstrain

In [ ]:
henergy, hstress = ???

#### Import reference model solution

In [ ]:
file = './data/3D_PlaneStrain_Elasticity_CSMA_inference'
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)

strain_ref,svars_ref,stress_ref,energy_ref,dissipationDot_ref = data

#### Compare results

Plot the stress vs strain history as predicted by the model and compare it with the one of the reference model

In [ ]:
plt.plot(hstrain[:,umat_dir],
         hstress[:,umat_dir],
         color = 'darkblue',
         alpha=1,linewidth=1, marker='o',markersize=5,label='TANN')
plt.plot(strain_ref[:,umat_dir],
         stress_ref[:,umat_dir],
         color='darkorange',
         alpha=1,linewidth=2, marker='s',markersize=0,label='ref')
plt.xlabel("$\\varepsilon$")
plt.ylabel("$\sigma=\partial_{\\varepsilon} NN(\\varepsilon)$")
plt.legend()
plt.show()

Plot the free energy vs strain history as predicted by the model and compare it with the one of the reference model

In [ ]:
plt.plot(hstrain[:,umat_dir],
         henergy,
         color = 'darkblue',
         alpha=1,linewidth=1, marker='o',markersize=5,label='TANN')

plt.plot(strain_ref[:,umat_dir],
         energy_ref,
         color = 'coral',
         alpha=1,linewidth=2, marker='s',markersize=0,label='ref')
plt.xlabel("$\\varepsilon$")
plt.ylabel("$\psi=NN(\\varepsilon)$")
plt.legend()
plt.show()

#### Compute and plot the error with respect to the reference constitutive model

In [ ]:
MAE_energy = (np.abs(energy_ref-henergy[:,0]))
MAE_stress = (np.abs(stress_ref[:,umat_dir]-hstress[:,umat_dir]))

In [ ]:
plt.plot(hstrain[:,umat_dir],
         MAE_energy,
         alpha=1,linewidth=1, marker='o',markersize=4,label='$MAE_{\psi}$')

plt.plot(hstrain[:,umat_dir],
         MAE_stress,
          alpha=1,linewidth=1, marker='s',markersize=4,label='$MAE_{\sigma}$')
plt.xlabel("$\\varepsilon$")
plt.ylabel("Mean Absolute Error")
plt.legend()
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b33f54f1-f98d-4fed-99c0-2484f115edd9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>